## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,62.06,54,75,8.05,broken clouds
1,1,Kaitangata,NZ,-46.2817,169.8464,51.85,88,100,3.36,overcast clouds
2,2,Oussouye,SN,12.4850,-16.5469,75.61,89,99,1.83,overcast clouds
3,3,Bang Rakam,TH,16.7585,100.1174,87.80,62,20,5.75,few clouds
4,4,Dikson,RU,73.5069,80.5464,33.08,74,100,5.50,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
print("What is your minimum temperature preference?")
min_temp = float(input("min_temp: "))

print("What is your maximum temperature preference?")
max_temp = float(input("max_temp: "))


What is your minimum temperature preference?
min_temp: 50
What is your maximum temperature preference?
max_temp: 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,62.06,54,75,8.05,broken clouds
1,1,Kaitangata,NZ,-46.2817,169.8464,51.85,88,100,3.36,overcast clouds
2,2,Oussouye,SN,12.4850,-16.5469,75.61,89,99,1.83,overcast clouds
3,3,Bang Rakam,TH,16.7585,100.1174,87.80,62,20,5.75,few clouds
6,6,Saint-Philippe,RE,-21.3585,55.7679,74.79,80,37,14.63,light rain


In [17]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

553


City_ID                553
City                   553
Country                548
Lat                    553
Lng                    553
Max Temp               553
Humidity               553
Cloudiness             553
Wind Speed             553
Current Description    553
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

548


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,62.06,54,75,8.05,broken clouds
1,1,Kaitangata,NZ,-46.2817,169.8464,51.85,88,100,3.36,overcast clouds
2,2,Oussouye,SN,12.4850,-16.5469,75.61,89,99,1.83,overcast clouds
3,3,Bang Rakam,TH,16.7585,100.1174,87.80,62,20,5.75,few clouds
6,6,Saint-Philippe,RE,-21.3585,55.7679,74.79,80,37,14.63,light rain


In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hobart,AU,62.06,broken clouds,-42.8794,147.3294,
1,Kaitangata,NZ,51.85,overcast clouds,-46.2817,169.8464,
2,Oussouye,SN,75.61,overcast clouds,12.4850,-16.5469,
3,Bang Rakam,TH,87.80,few clouds,16.7585,100.1174,
6,Saint-Philippe,RE,74.79,light rain,-21.3585,55.7679,
8,Sao Gotardo,BR,65.14,scattered clouds,-19.3111,-46.0489,
9,Kodiak,US,54.91,overcast clouds,57.7900,-152.4072,
10,Cururupu,BR,76.73,overcast clouds,-1.8283,-44.8683,
11,Samarai,PG,80.76,overcast clouds,-10.6167,150.6667,
12,Fairbanks,US,50.09,moderate rain,64.8378,-147.7164,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
H

In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hobart,AU,62.06,broken clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
1,Kaitangata,NZ,51.85,overcast clouds,-46.2817,169.8464,Kaitangata Motor Camp
2,Oussouye,SN,75.61,overcast clouds,12.4850,-16.5469,Auberge du Routard
3,Bang Rakam,TH,87.80,few clouds,16.7585,100.1174,เมญ่า&เนย
6,Saint-Philippe,RE,74.79,light rain,-21.3585,55.7679,Le four à pain
8,Sao Gotardo,BR,65.14,scattered clouds,-19.3111,-46.0489,HOTEL BOMTEMPO
9,Kodiak,US,54.91,overcast clouds,57.7900,-152.4072,Best Western Kodiak Inn
10,Cururupu,BR,76.73,overcast clouds,-1.8283,-44.8683,POUSADA TRAVÉS'CIA
11,Samarai,PG,80.76,overcast clouds,-10.6167,150.6667,Nuli Sapi
12,Fairbanks,US,50.09,moderate rain,64.8378,-147.7164,Westmark Fairbanks Hotel & Conference Center


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))